In [1]:
import pandas as pd
import pickle5 as pickle
import numpy as np
import seaborn as sns
from collections import defaultdict

import matplotlib.pyplot as plt

import plot

In [2]:
palette = plot.get_categorical_palette()
heuristic = 'Path score heuristic, 0.95'

In [3]:
base_dir = '/Users/rhodos/Desktop/Projects/akbc2021/'
figure_dir = f'{base_dir}figures/figures_for_paper/'
results_dir = f'{base_dir}explanations/linklogic/for_paper/true_false_nonsense/'

kge_thresh = 0.95
log_thresh = -np.log(1-kge_thresh)

In [11]:
data = pd.read_csv(f'{results_dir}/parsimony_analysis.csv', sep='\t', index_col=0)

In [14]:
data['Exp'] = data['Exp'].replace({'nonsense': 'Nonsense', 'false_facts': 'False Fact', 'true_facts': 'True Fact'})
data.columns = ['Query Triple Category', 'Query Triple', 'KGE Score', 'linklogic Fidelity', 'Total Features', 'Explanation Size']

In [17]:
data.head()

,Query Triple Category,Query Triple,KGE Score,linklogic Fidelity,Total Features,Explanation Size
0,Nonsense,"['josaphat_kuncevyc', 'nationality', 'george_e...",0.013099,-0.000271,109,0
1,Nonsense,"['blanka_of_namur', 'gender', 'friedrich_vii_m...",0.024888,-0.000198,151,0
2,Nonsense,"['alpheus_baker', 'profession', 'martha_bulloch']",0.019042,-0.001908,188,0
3,Nonsense,"['tim_russert', 'ethnicity', 'davis_hanson_wai...",0.022815,-0.000209,188,0
4,Nonsense,"['andy_bowen', 'place_of_birth', 'kadambini_ga...",0.038236,-0.000322,139,0


In [ ]:
# Get metrics and organize into dataframe
triples, expt, kge_score, fidelity, no_of_features, num_explanations_linklogic, num_explanations_heuristic = [], [], [], [], [], [], []
selected_heuristic_paths = defaultdict(list)
for exp in ['Nonsense', 'False Fact', 'True Fact']:
    selected_heuristic_paths[exp] = list()
    for d in data[exp]:
        expt.append(exp)
        triples.append(d["query_triple"])
        kge_score.append(float(d["query_triple_kge_score"]))
        fidelity.append(d["linklogic_metrics"]["test_acc"])
        num_explanations_linklogic.append(sum([1 if f["coef"] !=0.0 else 0 for f in d["linklogic_features"]]))
        num_heuristic_paths = sum([1 for f in d['linklogic_features'] if f['kge_score']['path_score'] > log_thresh])
        num_explanations_heuristic.append(num_heuristic_paths)
        
df = pd.DataFrame({"Query Triple Category": expt, 
              "Triple": triples,
              "KGE Score for Query Triple": kge_score,
              "linklogic Explanation Fidelity": fidelity,
              "linklogic": num_explanations_linklogic,
              heuristic: num_explanations_heuristic,
              })

In [ ]:
id_vars = ['Triple', 'Query Triple Category']
df1 = df[id_vars + ['KGE Score for Query Triple', 'linklogic Explanation Fidelity']]
mdata = df1.melt(id_vars=id_vars)

g = sns.FacetGrid(mdata, col="variable", sharey=False)
_ = g.map_dataframe(sns.boxplot, x="Query Triple Category", y="value", boxprops=dict(alpha=.5), color=palette['generic'])
_ = g.set_xticklabels(size=10)
_ = g.set_titles('')
_ = g.axes[0,0].set_ylabel('KGE Score')
_ = g.axes[0,1].set_ylabel('linklogic Fidelity')
g.savefig(f'{figure_dir}Parsimony_A_B.{plot.figtype}', bbox_inches='tight')

In [ ]:
plt.figure(figsize=(5,4))
df2 = df[id_vars + ["linklogic", heuristic]].melt(id_vars=id_vars)
g = sns.boxplot(data=df2, x="Query Triple Category", y="value", hue="variable", 
            palette={'linklogic': palette['linklogic'], heuristic: palette['Heuristic95']},
            boxprops=dict(alpha=.9))
_ = g.set(xlabel=None, ylabel='Explanation Size')
_ = g.legend(title='')
g.get_figure().savefig(f'{figure_dir}Parsimony_C.{plot.figtype}', bbox_inches='tight')

In [ ]:
# TODO:
# Make panel B linklogic blue
# Adjust ylim in panel B to max 30